Before use, it is needed to install pytrends library:

`pip install pytrends`

Documentation: https://pypi.org/project/pytrends/

# Google Trends Scrapper

In [150]:
import pandas as pd
from pytrends.request import TrendReq
import time

def query_google_trends(terms_dict):
    
    # Verify that terms_dict is a dictionary
    if not isinstance(terms_dict, dict):
        raise ValueError("The 'terms_dict' parameter must be a dictionary.")

    # Verify that the dictionary of terms has a maximum of 5 terms
    if len(terms_dict) > 5:
        raise ValueError("The dictionary of terms cannot have more than 5 terms.")

    
    # Configure the connection to Google Trends
    pytrends = TrendReq(hl='en-US', tz=360)
    

    # Define the search parameters
    kw_list = list(terms_dict.values()) # Usar los valores del diccionario como términos de búsqueda
    cat = 0  # All categories
    timeframe = '2019-01-01 2022-12-31'  # Date range
    geo = 'US'  # Geographic area: United States
    gprop = ''  # gprop must be empty (to indicate web), images, news, youtube, or froogle (google shopping)

    # Build the payload for the query
    pytrends.build_payload(kw_list, cat=cat, timeframe=timeframe, geo=geo, gprop=gprop)

    # Query the interest data over time
    interest_over_time_df = pytrends.interest_over_time()
    
    # Si la busqueda no devuelve resultados, crear un dataframe con todo ceros para todas las semanas
    if interest_over_time_df.empty:
        # Obtener todas las semanas en el rango especificado
        date_range = pd.date_range(start='2019-01-01', end='2022-12-31', freq='W')
        # Crear un DataFrame con todas las semanas y valores 0
        interest_over_time_df = pd.DataFrame(0, index=date_range, columns=list(terms_dict.keys()))

    else:
        # Drop isPartial column from the dataframe
        if 'isPartial' in interest_over_time_df.columns:
            interest_over_time_df = interest_over_time_df.drop('isPartial', axis=1)
        
        # Renombrar las columnas con las claves del diccionario
        interest_over_time_df.columns = list(terms_dict.keys())

    # Return the DataFrame
    return interest_over_time_df

#### Format export csv

In [151]:
def export_csv(df_in, export_name):
    
    # Agregar la columna 'year_week'
    df_in['year_week'] = df_in.index.strftime('%Y-%U')
    
    # Utiliza la función melt para reorganizar el DataFrame
    melted_df = pd.melt(df_in, id_vars=['year_week'], var_name='airline', value_name=export_name)
    
    melted_df.to_csv(export_name + '.csv', index=False)

# Usage

In [159]:
# Key: Airline name as appearing on train_df
# Value: term to search on Google Trends

aerolineas_dict = {
    'Southwest Airlines Co.': 'Southwest Airlines',
     'Delta Air Lines Inc.': 'Delta Air Lines',
     'GoJet Airlines, LLC d/b/a United Express': 'GoJet Airlines',
     'Republic Airlines': 'Republic Airlines',
     'Capital Cargo International' : 'Capital Cargo International',
     'Envoy Air': 'Envoy Air',
     'Empire Airlines Inc.': 'Empire Airlines',
     'United Air Lines Inc.': 'United Air Lines',
     'Comair Inc.': 'Comair',
     'SkyWest Airlines Inc.': 'SkyWest Airlines',
     'American Airlines Inc.': 'American Airlines',
     'Horizon Air': 'Horizon Air',
     'Compass Airlines': 'Compass Airlines',
     'Alaska Airlines Inc.': 'Alaska Airlines',
     'JetBlue Airways': 'JetBlue Airways',
     'Mesa Airlines Inc.': 'Mesa Airlines',
     'Endeavor Air Inc.': 'Endeavor Air',
     'Frontier Airlines Inc.': 'Frontier Airlines',
     'ExpressJet Airlines Inc.': 'ExpressJet Airlines',
     'Commutair Aka Champlain Enterprises, Inc.': 'Commutair',
     'Allegiant Air': 'Allegiant Air',
     'Spirit Air Lines': 'Spirit Air Lines',
     'Air Wisconsin Airlines Corp': 'Air Wisconsin',
     'Peninsula Airways Inc.': 'Peninsula Airways',
     'Hawaiian Airlines Inc.': 'Hawaiian Airlines',
     'Trans States Airlines': 'Trans States Airlines'
}

# Crear un nuevo diccionario con los valores modificados
aerolineas_dict_strike = {key: value + ' strike' for key, value in aerolineas_dict.items()}

# Crear un nuevo diccionario con los valores modificados
aerolineas_dict_cancellations = {key: value + ' cancellations' for key, value in aerolineas_dict.items()}

## TRENDS COMPARED (UP TO 5 TOPICS)

In [155]:
# List of terms to search in Google Trends (with more than 5 terms)
terms_to_search = {
    'Southwest Airlines Co.': 'Southwest Airlines',
     'Delta Air Lines Inc.': 'Delta Air Lines',
     'GoJet Airlines, LLC d/b/a United Express': 'GoJet Airlines',
     'Republic Airlines': 'Republic Airlines'}

# Call the function to query trends (COMPARED)

df_result = query_google_trends_test(terms_to_search)
df_result

,Southwest Airlines Co.,Delta Air Lines Inc.,"GoJet Airlines, LLC d/b/a United Express",Republic Airlines
date,,,,
2019-01-06,42,0,0,0
2019-01-13,39,0,0,0
2019-01-20,37,0,0,0
2019-01-27,39,0,0,0
2019-02-03,37,0,0,0
...,...,...,...,...
2022-11-27,37,0,0,0
2022-12-04,32,0,0,0
2022-12-11,30,0,0,0


In [156]:
#export_csv(df_result, 'prueba')

## TRENDS COMPARED (more than 5) (WIP)

In [157]:
# Compare in pairs to obtain the maximum
terms_to_search = aerolineas_dict

topic_aux = list(terms_to_search.values())[0]

for topic in list(terms_to_search.values()):
    if topic == list(terms_to_search.values())[0]:
        continue
        
    pair_to_search = [topic_aux, topic]

    df_result = query_google_trends(pair_to_search)
    
    # Calculate the maximum and get the column name where it occurs
    max_column_name = df_result.max().idxmax()
    
    topic_aux = max_column_name
    
    # Agregar un retraso de 2 segundos entre las solicitudes para evitar timeout
    time.sleep(2)
    

# Set most popular term    
most_popular_term = topic_aux
print(f"Column name with max value: {most_popular_term}")

ValueError: The 'terms_dict' parameter must be a dictionary.

In [158]:
# Now that we have the most popular, we are going to obtain from google trends data for the rest of the terms,
# always comparing with the most popular, so that we can aggregate all the values and they make sense

# Remove most popular term from list of terms to search

terms_to_search_compared = terms_to_search.copy()
terms_to_search_compared.remove(most_popular_term)

# Make groups of 4 (as +1 with the most popular term it makes 5 and we can only search up to 5 at the same time)

# Lista para almacenar las sublistas
sublists = []

# Inicializar una sublista vacía
current_sublist = []

# Iterar a través de los elementos de la lista original
for term in terms_to_search_compared:
    
    # Agregar el término actual a la sublista actual
    current_sublist.append(term)
    
    # Si la sublista alcanza el número máximo de elementos, agregarla a la lista de sublistas
    if len(current_sublist) == 4:
        sublists.append(current_sublist)
        # Inicializar una nueva sublista vacía
        current_sublist = []

# Agregar cualquier sublista restante
if current_sublist:
    sublists.append(current_sublist)
    
# Imprimir las sublistas resultantes
for i, sublist in enumerate(sublists, start=1):
    print(f"Sublista {i}: {sublist}")

AttributeError: 'dict' object has no attribute 'remove'

In [10]:
# Now we will search in groups of 5 from Google Trends:

# Initialize an empty list to store individual DataFrames
dfs_result_sublist = []

for sublist in sublists:
    lista = sublist.copy()
    lista.append(most_popular_term)

    results_sublist = query_google_trends(lista)
    
    # Append the DataFrame to the list
    dfs_result_sublist.append(results_sublist)
    
    # Agregar un retraso de 2 segundos entre las solicitudes para evitar timeout
    time.sleep(2)

In [11]:
# Concatenar resultados en un DataFrame
concatenated_df = pd.concat(dfs_result_sublist, axis=1)

# Remove duplicate columns (para el most_popular_term)
concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.duplicated()]

In [12]:
concatenated_df

,Delta Air Lines cancellations,American Airlines cancellations,SkyWest Airlines cancellations,United Air Lines cancellations,Southwest Airlines cancellations,Republic Airlines cancellations,Envoy Air cancellations,Endeavor Air cancellations,Comair cancellations,JetBlue Airways cancellations,...,Capital Cargo International cancellations,Air Wisconsin cancellations,ExpressJet cancellations,Hawaiian Airlines cancellations,GoJet Airlines cancellations,Commutair cancellations,Compass Airlines cancellations,Trans States Airlines cancellations,Empire Airlines cancellations,Peninsula Airways cancellations
date,,,,,,,,,,,,,,,,,,,,,
2019-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-01-13,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-01-20,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-01-27,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-02-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-12-11,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
export_csv(concatenated_df, 'gt_airlines_cancellations_compared')

## INDIVIDUAL TRENDS (no limit)

In the individual trends, each topic will have a value of '100' for the week when there were more google 
searches in the period that we are analysing. 
A value of 100 for topic 1 can mean 500,000 searches on that week, while for topic 2 it can mean 2,000 searches,
this just means that topic 1 is way more popular than topic 2, but we are indexing individually.

So that we don't get zeros for all weeks in less searched terms in compared to more popular terms.

In [166]:
# List of terms to search in Google Trends (with more than 5 terms)
terms_to_search = aerolineas_dict
# Initialize an empty list to store individual DataFrames
dfs_result_list = []

# Call the function to query trends (INDIVIDUALLY)
for key, topic in terms_to_search.items():
    nuevo_diccionario = {key: topic}
    
    df_result_ind = query_google_trends(nuevo_diccionario)
    
    # Append the DataFrame to the list
    dfs_result_list.append(df_result_ind)

In [167]:
concatenated_df = pd.concat(dfs_result_list, axis=1)

In [168]:
export_csv(concatenated_df, 'gt_airlines_individual')